# Sumy uwierzytelniające wiadomość, [Message Authentication Codes (MAC)](https://en.wikipedia.org/wiki/Message_authentication_code)

## [HMAC](https://en.wikipedia.org/wiki/HMAC) (*Hash based MAC*)

Ze względu na [atak rozszerzający](https://en.wikipedia.org/wiki/Length_extension_attack)(ang. *Length Extension Attacks*) funkcje skrótu zbudowane w oparciu o paradygmat [Merkle–Damgård (MD)](https://en.wikipedia.org/wiki/Merkle%E2%80%93Damg%C3%A5rd_construction) nie mogą być wykorzystane **wprost** do wyznaczania kodów uwierzytalniających i wymagają korzystania z konstrukcji [HMAC](https://en.wikipedia.org/wiki/HMAC).

In [4]:
import binascii
from Crypto.Hash import HMAC, SHA256
key = b"tojesthaslo"
hmac = HMAC.new(key, digestmod=SHA256)
hmac.update(b'Salve mundi, ')
hmac.update(b'Hello world, ')
hmac.update("Po łacinie wszystko brzmi mądrze!".encode('utf-8'))
mac = hmac.digest()
print("MAC=",binascii.hexlify(mac))

MAC= b'609e799f65db3558e59c78a4dae85e37463fadee2d44e9d945a4d02de0d8c12d'


Weryfikacja przebiegga wg schematu

In [5]:
key = b"tojesthaslo"
msg = b'Salve mundi, ' + b'Hello world, ' + "Po łacinie wszystko brzmi mądrze!".encode('utf-8')
hmac_verifier = HMAC.new(key, digestmod=SHA256)
hmac_verifier.update(msg)
try:
    hmac_verifier.verify(mac)
    print("The message '%s' is authentic" % msg.decode('utf-8'))
except ValueError:
    print("The message or the key is wrong")

The message 'Salve mundi, Hello world, Po łacinie wszystko brzmi mądrze!' is authentic


## KMAC (Keccak MAC)

Algorytmy Keccak oraz wywiedziony z niego [SHA3](https://en.wikipedia.org/wiki/SHA-3) korzystają z paradygmatu gąbki i nie są podatne na atak rozszerzający. Wobec tego sumy uwierzytelniające mogą być za jego pomoca obliczane wprost $MAC=SHA3(\, key\, ||\, message\, )$ gdzie operator '$||$' oznacza sklejanie ciągów bitów/bajtów.

"*Unlike SHA-1 and SHA-2, Keccak does not have the length-extension weakness, hence does not need the HMAC nested construction. Instead, MAC computation can be performed by simply prepending the message with the key.*" patrz [Keccak (SHA3) website](https://keccak.team/index.html)

In [7]:
from Crypto.Hash import SHA3_256
import binascii
hasher = SHA3_256.new() 
key = b"tojesthaslo"
msg = b'Salve mundi, ' + b'quidquid discis, tibi discis, ' + "Po łacinie wszystko brzmi mądrze!".encode('utf-8')
hasher.update(key+msg)
sha3_256_hash = hasher.digest()
print("hash =", binascii.hexlify(sha3_256_hash))

hash = b'439583db2fa4427b290248aed5af3529da2feb58962533640a9a549b9ae3108b'


## Inne typy sum kontronych

Inne typy sum kontrolnych zostaną omówione przy okazji szyfrowania uwierzytelnionego: CMAC (Cipher-based MAC), Poly1305 (Bernstein's one-time authenticator), GMAC (Galois MAC), SipHash (simple, fast, secure MAC).